In [1]:
!python --version

Python 3.7.5


In [2]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [3]:
from bert import extract_features
from bert import tokenization

In [4]:
VOCAB_FILE_LOCATION='../uncased_L-12_H-768_A-12/vocab.txt'

tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE_LOCATION)

In [5]:
examples = extract_features.read_examples('./data.txt')

In [6]:
features = extract_features.convert_examples_to_features(examples=examples, seq_length=25, tokenizer=tokenizer)


INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] the quick brown fox jumps over the lazy dog . [SEP]
INFO:tensorflow:input_ids: 101 1996 4248 2829 4419 14523 2058 1996 13971 3899 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [7]:
model_fn = extract_features.model_fn_builder(
    bert_config='../uncased_L-12_H-768_A-12/bert_config.json',
    init_checkpoint='../uncased_L-12_H-768_A-12/bert_model.ckpt',
    layer_indexes=[-1, -2, -3, -4],
    use_tpu=False,
    use_one_hot_embeddings=False
)

In [8]:
run_config = tf.contrib.tpu.RunConfig(
    master=None,
    tpu_config=tf.contrib.tpu.TPUConfig(
        num_shards=8,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
    )
)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [9]:
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False,
    model_fn=model_fn,
    predict_batch_size=32,
    config=run_config
)

INFO:tensorflow:Using config: {'_model_dir': '/var/folders/1h/kbc1m98s7434dpfydry4x8140000gn/T/tmp6cjm1ryi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1303ec050>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_

In [10]:
input_fn = extract_features.input_fn_builder(
    features=features,
    seq_length=128
)

In [11]:
for result in estimator.predict(input_fn, yield_single_examples=True):
    print(result)

INFO:tensorflow:Could not find trained model in model_dir: /var/folders/1h/kbc1m98s7434dpfydry4x8140000gn/T/tmp6cjm1ryi, running initialization to predict.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
ERROR:tensorflow:Error recorded from prediction_loop: 'str' object has no attribute 'hidden_dropout_prob'
INFO:tensorflow:prediction_loop marked as finished


AttributeError: 'str' object has no attribute 'hidden_dropout_prob'